In [15]:
using LinearAlgebra

Problem 3

In [14]:
function find_energy_minimum(A::Matrix{Float64}, b::Vector{Float64}, c::Float64)
    # Solve the linear system Ax = -b
    x = A \ (-b)
    
    # Calculate the minimum energy value
    energy = 0.5 * dot(x, A * x) + dot(b, x) + c
    
    return x, energy
end


function find_energy_minimum_safe(A::Matrix{Float64}, b::Vector{Float64}, c::Float64)
    # Check if matrix is square
    n, m = size(A)
    if n != m
        throw(ArgumentError("Matrix A must be square"))
    end
    
    # Check if matrix is symmetric
    if !isapprox(A, A', rtol=1e-8)
        throw(ArgumentError("Matrix A must be symmetric"))
    end
    
    # Check if matrix is positive definite
    # Try Cholesky factorization - it only works for SPD matrices
    try
        cholesky(A)
    catch e
        throw(ArgumentError("Matrix A must be positive definite"))
    end
    
    # If we get here, A is SPD, so solve the system
    x = A \ (-b)
    energy = 0.5 * dot(x, A * x) + dot(b, x) + c
    
    return x, energy
end

function create_test_system(n::Int)
    # Create tridiagonal matrix for efficiency
    # The matrix will represent the discretized version of the energy
    # E(x^i) = Σᵢ (x^i + x^(i-1) + sin(πi/n))²
    
    # Initialize matrix
    A = zeros(n, n)
    
    # Fill the matrix
    for i in 1:n
        if i > 1
            A[i,i-1] = 1.0  # x^(i-1) term
            A[i-1,i] = 1.0  # Symmetry
        end
        A[i,i] = 1.0  # x^i term
    end
    
    # Create the full matrix A'A which represents the quadratic form
    A = A'A
    
    # Create vector b
    b = zeros(n)
    for i in 1:n
        s = sin(π * i / n)
        if i > 1
            b[i] += s
        end
        b[i] += s
        if i < n
            b[i] += s
        end
    end
    
    # Double b because of the square in the energy function
    b = 2 * b
    
    # c is the sum of squares of sin terms (constant part)
    c = sum(sin(π * i / n)^2 for i in 1:n)
    
    return A, b, c
end


create_test_system (generic function with 1 method)

In [13]:

# Test the system
n = 10^3
println("Creating test system...")
@time A, b, c = create_test_system(n)

println("Solving system...")
@time x, energy = find_energy_minimum_safe(A, b, c)

println("Energy at minimum: ", energy)
println("First few components of solution:")
println(x[1:5])

Creating test system...
  0.020163 seconds (7 allocations: 15.275 MiB, 33.37% gc time)
Solving system...
  0.022249 seconds (12 allocations: 22.920 MiB)
Energy at minimum: -500.0461108189077
First few components of solution:
[6.289465050821832, -6.283181875850016, -0.025132699887148984, 6.28318206188733, -6.270615153833876]
